In [1]:
#verification that ca-bc < 2ta yileds a unique equilibrium because the unconstrained solution is valid

#Core scientific modules
import numpy as np
from scipy.stats import uniform, norm
def perc_75(x): return np.percentile(x,75) #Percentile calculations
def perc_25(x): return np.percentile(x,25)

#Pandas, fileIO
import os, glob
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = 100

#Plotting, display
from IPython.display import display, Markdown
from matplotlib import pyplot as plt, rcParams
from cycler import cycler
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
sns.set(style = "white", context="paper")
notebook_default_rcparams = { #Set default RC parameters
    "axes.labelsize": 30,
    "legend.fontsize":24,
    "xtick.labelsize":28,
    "ytick.labelsize":28,
    "axes.grid":True,
    "legend.framealpha":0.5,
    "lines.linewidth":5,
    "legend.loc":'upper left'}
rcParams.update(notebook_default_rcparams)
pltsettings ={'color':'blue', 'linestyle':'-','marker':'.','alpha':0.5}

In [9]:
# Data for plotting

# #########region ca-cb < 2ta
# ca    = 0
# cb    = 0
# ta    = 3
# paa_analytical = 0.33*(2*ca+cb+2*ta)
# pba_analytical = 0.33*(2*cb+ca+ta)

#########region ca-cb > 2ta
ca    = 8
cb    = 1
ta    = 3
paa_analytical = ca
pba_analytical = ca - ta



dist  = 'uniform'
maxpx = ca+5
npts = 100
paa_arr = np.linspace(ca,maxpx,npts)
pba_arr = np.linspace(cb,maxpx,npts)
obja = np.zeros((paa_arr.size,pba_arr.size))
objb = np.zeros((paa_arr.size,pba_arr.size))
constraintmat = np.zeros((paa_arr.size,pba_arr.size))

def get_xi_objects(dist='normal'):
    if dist=='uniform':
        return (np.arange(0.0, 1.0, 0.1),uniform.cdf,uniform.pdf)
    return (np.arange(-1.0, 1.0, 0.1),norm.cdf,norm.pdf)
def constraints(paa,pba,ca,cb,ta):
    if paa >= ca and pba >= cb and (0 <= paa-pba) and  (paa-pba <= ta):
        return True
    else:
        return False

In [10]:
_,F,f = get_xi_objects(dist)
for i,paa in enumerate(paa_arr):
    for j,pba in enumerate(pba_arr):
        if constraints(paa,pba,ca,cb,ta):
            constraintmat[i,j] = 1 
            obja[i,j] = (paa-ca)*(1-F((paa-pba)/ta))
            objb[i,j] = (pba-cb)*F((paa-pba)/ta)

In [11]:
print('analytical prices:',paa_analytical,pba_analytical)
obja_analytical = (paa_analytical-ca)*(1-F((paa_analytical-pba_analytical)/ta))
objb_analytical = (pba_analytical-cb)*F((paa_analytical-pba_analytical)/ta)
print('objective values:',obja_analytical,objb_analytical)
#constraints(paa_analytical,pba_analytical,ca,cb,ta)

analytical prices: 8 5
objective values: 0.0 4.0


In [12]:
from dsGameSolver.gameSolver import dsSolve

payoffMatrices = [np.array([obja,objb])]
equilibrium = dsSolve(payoffMatrices)
eq_indices = np.argmax(equilibrium['strategies'][0],axis=1)
print('computed prices:',paa_arr[eq_indices[0]],pba_arr[eq_indices[1]])
print('computed values:',np.round(equilibrium['stateValues'],3))

computed prices: 8.1010101010101 5.242424242424242
computed values: [[0.005 4.044]]
